# 0. Meta

## 0.1. Packages

## 0.2. Functions

# 1. Data import

# 2. Initial Data exploration

## 2.1. Structure of Data

## 2.2. Scale of Measure

## 2.2. Missing Entries

## 2.3. Categorial Variables

### 2.3.1. Counts per categorical level

### 2.3.2. Target distribution by categorical levels

## 2.4. Continuous Variables

### 2.4.1. Correlation

### 2.4.2. Linearity

### 2.4.3. Distribution

### 2.4.4. Outliers